<a href="https://colab.research.google.com/github/bchiramani/TraitementImages/blob/main/TP1TraitementImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import array
from random import *
import pandas as pd
import matplotlib.pyplot as plt
from math import log10, sqrt
from PIL import Image
from matplotlib import cm

# Read a PGM File

In [12]:
def read_pgm(path):

  with open(path, 'r') as f :
    pgm = f.read()
  print(type(pgm))
  image = np.array([x.split() for x in np.array(pgm.split('\n'))[4:]])
  print(type(image))
  return image.astype('int32')

## Explore some caracteristics


*   the mean
*   the standard deviation
*   Histrogram
*   Cumulated histogram



In [13]:
def display_histogram(tab, title=''):
  index = range(0, 256)
  plt.figure(figsize=(9, 3))
  plt.subplot(131)
  plt.bar(x=index, height=tab);
  plt.title(title)

In [14]:
def calculate_mean_std(image):
  sum = 0
  height = image.shape[0]
  width = image.shape[1]
  sum = image.sum()
  mean = sum / (height * width)
  sum = 0
  for h in range(height):
    for w in range(width):
      sum += (int(image[h][w]) - mean) ** 2
  std = (sum / (height * width)) ** (1/2)
  return mean , std

In [15]:
def histogram(image):
  height = image.shape[0]
  width = image.shape[1]
  tab = np.zeros((256))
  for h in range(height):
    for w in range(width):
      tab[int(image[h][w])] += 1
  return tab

In [16]:
def cumulated_histogram(image):
  tab = histogram(image)
  for i in range(1, 256):
    tab[i] += tab[i-1]
  return tab

In [17]:
def generate_histograms(image):

  mean , std = calculate_mean_std(image)
  print("The mean value is {} and the standard deviation is {}\n".format(mean, std))
  simple_hist = histogram(image)
  # display_histogram(simple_hist)
  cumulated_hist = cumulated_histogram(image)
  # display_histogram(cumulated_hist)
  return simple_hist, cumulated_hist

In [18]:
out = read_pgm('/content/chat.pgm')
print(out.dtype)

<class 'str'>
<class 'numpy.ndarray'>
int32


# Histogram equalization

In [19]:
def probability(image):
  
  height = image.shape[0]
  width = image.shape[1]
  prob_hist = histogram(image) / (height * width)
  return prob_hist

In [20]:
def cumulated_probability(image):
  height = image.shape[0]
  width = image.shape[1]
  cumulated_prob = cumulated_histogram(image) / (height * width)
  
  return cumulated_prob

In [21]:
def function_A(image):
  
  return cumulated_probability(image) * 255

In [22]:
def write_pgm(height, width, filename, matrice):
  data = ""
  for i in range(0, height):
    for j in range(0, width):
      data += str(matrice[i][j]) + " "
    data +="\n"
  fout = open(filename, "w")
  pgm_header = 'P2' + '\n' + str(width) + ' ' + str(height) + ' ' + str(255) + '\n'
  fout.write(pgm_header)
  fout.write(data)
  fout.close()


In [23]:
def equalize_histogram(hist, image):

  equalized_hist = np.zeros((256))
  equalized_image = []
  sum = hist[0]
  height = image.shape[0]
  width = image.shape[1]  

  n1 = function_A(image).astype('int32')

  for i in range(1, 255):
    if n1[i] == n1[i-1]:
      sum += hist[i]
    else:
      equalized_hist[n1[i-1]] = sum
      sum = hist[i]


  display_histogram(hist, 'histogram before equalization')
  display_histogram(equalized_hist, 'histogram after equalization')



  for h in range(0, height):
    row = []
    for w in range(0, width):
      row.append(n1[image[h][w]])
    equalized_image.append(row)
    
  return(equalized_image)


In [24]:
def equalize_image(input_path, output_path):
  image = read_pgm(input_path)
  simple_hist, _ = generate_histograms(image)
  matrice = equalize_histogram(simple_hist, image)
  write_pgm(image.shape[0] ,image.shape[1] , output_path, matrice)

## Linear Transformation

In [25]:
def calcul_coef(x1, y1, x2, y2):
  a = (y2 - y1) / (x2 - x1)
  b = y1 - a * x1
  return a, b

In [26]:
def transformation_lineaire(x1, y1, x2, y2):
  input = range(0, 256)
  transformed = np.zeros((256))
  a0, b0 = calcul_coef(0, 0, x1, y1)
  a1, b1 = calcul_coef(x1, y1, x2, y2)
  a2, b2 = calcul_coef(x2, y2, 255, 255)


  transformed[0: x1] = np.array([a0 * x + b0 for x in input[0: x1]])
  transformed[x1: x2] = np.array([a1 * x + b1 for x in input[x1: x2]])
  transformed[x2: 255] = np.array([a2 * x + b2 for x in input[x2: 255]])

  return transformed

In [27]:
def linear_tranformation(input_path, ouput_path):

  image = read_pgm(input_path)
  
  result = transformation_lineaire(50, 100, 150, 200)

  height = image.shape[0]
  width = image.shape[1]
  transformed_image = []

  for h in range(0, height):
    row = []
    for w in range(0, width):
      row.append(result[image[h][w]])
    transformed_image.append(row)
    
  write_pgm(height, width, ouput_path, transformed_image)

  return(transformed_image)


In [28]:
matrice = linear_tranformation("/content/chat.pgm", "/content/linear_chat.pgm")

<class 'str'>
<class 'numpy.ndarray'>


# Filters

In [29]:
def generate_nbr(pix):
  nbr = randint(0, 20)
  if nbr == 0:
    return 0
  if nbr == 20:
    return 255
  return pix

In [30]:
def bruit(image):
  height = image.shape[0]
  width = image.shape[1]
  for i in range(0, height):
    for j in range(0, width):
      image[i, j] = generate_nbr(image[i, j])
  return image


In [31]:
def add_padding(filtre,image):
  taille= filtre.shape[0]
  height = image.shape[0]
  width = image.shape[1]
  extended_image = np.zeros((height + filtre.shape[0], width + filtre.shape[1]))
  for i in range(0, image.shape[0]):
    for j in range(0, image.shape[1]):
      extended_image [int(filtre.shape[0] / 2) + i][int(filtre.shape[1] / 2) + j] = image[i][j]

  return extended_image

In [32]:
def apply_convolution(image, kernel):

    kernel = np.flipud(np.fliplr(kernel))    
    output = np.zeros_like(image)            
    
    image_padded = np.zeros((image.shape[0] + (kernel.shape[0]-1), 
                             image.shape[1] + (kernel.shape[1]-1)))   
    image_padded[(kernel.shape[0]//2):-(kernel.shape[0]//2), 
                 (kernel.shape[1]//2):-(kernel.shape[1]//2)] = image
    
    for x in range(image.shape[1]):     
        for y in range(image.shape[0]):
            
            output[y,x]=(kernel*image_padded[y:y + kernel.shape[0], x:x + kernel.shape[1]]).sum()
    return output

In [33]:
def apply_linear_filter(path, filtre, filter_name=''):
  
  image = read_pgm(path)
  # print("original image : ", image)
  image_bruit = bruit(image)
  # print("image bruitée : ", image_bruit) 
  write_pgm(image.shape[0], image.shape[1], "noise_image.pgm", image_bruit)
  image_convolue = apply_convolution(image_bruit, filtre)
  # print("image convolue avec ce filtre : ", image_convolue)
  write_pgm(image_convolue.shape[0], image_convolue.shape[1], '{}_image.pgm'.format(filter_name), image_convolue)

  return image_convolue

In [34]:
def apply_nonlinear_filter(path, kernel_size = (3, 3), filter_name="median"):
    image = read_pgm(path)
    image_bruit = bruit(image)
    write_pgm(image.shape[0], image.shape[1], "noise_image.pgm", image_bruit)

    output = np.zeros_like(image)            
    image_padded = np.zeros((image.shape[0] + (kernel_size[0]-1), 
                             image.shape[1] + (kernel_size[1]-1)))   
    
    image_padded[(kernel_size[0]//2):-(kernel_size[0]//2), 
                 (kernel_size[1]//2):-(kernel_size[1]//2)] = image

    for x in range(image.shape[1]):     
          for y in range(image.shape[0]):

              output[y,x] = np.median(image_padded[y:y + kernel_size[0], x:x + kernel_size[1]])

    write_pgm(output.shape[0], output.shape[1], '{}_image.pgm'.format(filter_name), output)
    return output

# Filtre moyenneur

In [35]:
def moyenneur(size):
  return np.ones((size, size)) / size ** 2

In [36]:
for k in range(3, 22, 2):
  filtre = moyenneur(k)
  filtered_image = apply_linear_filter("/content/chat.pgm", filtre, 'moyenneur{}'.format(k))

<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'str'>
<class 'numpy.ndarray'>


In [37]:
filtre = moyenneur(3)
image_filtered_moyenneur = apply_linear_filter("/content/chat.pgm", filtre, 'moyenneur{}'.format(3))

<class 'str'>
<class 'numpy.ndarray'>


# Filtre Median

In [38]:
image_filtered_median = apply_nonlinear_filter("/content/chat.pgm", (3,3), 'median')

<class 'str'>
<class 'numpy.ndarray'>


# Filtre Passe Haut

In [39]:
filtre = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
image_filtered_passe_haut = apply_linear_filter("/content/chat.pgm", filtre, 'passehaut')

<class 'str'>
<class 'numpy.ndarray'>


# Signal To Noise Ratio

In [40]:
def signaltonoise(original, filtered):
  mean, _ = calculate_mean_std(original)
  upper_part = np.sqrt(((original - mean) ** 2).sum())
  lower_part = np.sqrt(((original - filtered) ** 2).sum())
  return upper_part / lower_part

In [41]:
image = read_pgm('/content/chat.pgm')
stn_pass_haut = signaltonoise(image, image_filtered_passe_haut)
stn_median = signaltonoise(image, image_filtered_median)
stn_moyenneur = signaltonoise(image, image_filtered_moyenneur)

<class 'str'>
<class 'numpy.ndarray'>


In [42]:
print(stn_pass_haut)
print(stn_median)
print(stn_moyenneur)

0.2837281919415543
8.648409901790869
3.7099069950231858


## PPM

In [43]:
def read_ppm(path):  
  with open(path, 'r') as f :
    ppm = f.read()
  image = np.array([ values.split('\n')for values in np.array(ppm.split('\n'))[5:]])
  
# provisoire
  im=image[0][0].split()
  print("im : ",im)
  x=[tuple(im[i:i+3]) for i in range(0, len(im), 3)]

# find solution
  # rgb=[]
  # for im in image:
  #   im=im[0].split()
  #   x=np.array([tuple(im[i:i+3]) for i in range(0, len(im), 3)])
  #   rgb.append(x)





  return x 


In [44]:
rgb = read_ppm("/content/peppers.ppm")
print(rgb)


im :  ['62', '29', '64', '63', '31', '64', '63', '34', '64', '65', '30', '60', '66', '27', '59', '63', '31', '62', '61', '31', '62', '63', '30', '61', '63', '30', '59', '64', '31', '58', '64', '30', '61', '61', '30', '58', '65', '30', '56', '64', '32', '59', '61', '31', '59', '62', '30', '60', '63', '30', '62', '65', '32', '64', '65', '32', '61', '64', '29', '58', '61', '29', '57', '64', '30', '56', '65', '29', '58', '64', '30', '61', '66', '30', '61', '71', '31', '66', '67', '33', '67', '65', '33', '63', '64', '35', '62', '62', '32', '62', '68', '31', '64', '68', '35', '66', '70', '31', '65', '70', '31', '63', '71', '33', '65', '70', '34', '68', '68', '33', '67', '74', '37', '69', '74', '37', '69', '70', '36', '69', '70', '36', '70', '71', '37', '67', '71', '36', '67', '72', '34', '69', '74', '35', '70', '70', '34', '70', '69', '34', '67', '70', '35', '65', '67', '34', '63', '67', '32', '65', '67', '33', '66', '69', '37', '67', '69', '36', '66', '66', '35', '63', '67', '35', '64', '67

# Manual Threasholding

In [45]:
def thresholding(rgb,red,green,blue):
  result=[]
  for element in rgb:
    if int(element[0])>red:
      r=255
    else:
      r=0
    if int(element[1])>green:
      g=255
    else:
      g=0
    if int(element[2])>blue:
      b=255
    else:
      b=0
    x=tuple([r,g,b])
    result.append(x)
  print(result)


In [46]:
thresholding(rgb,100,20,30)

[(0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 255), (0, 255, 

In [47]:
def thresholding(rgb,red,green,blue,option):
  print(rgb)
  result=[]
  if option=="ET":
    for element in rgb:
      if((int(element[0])>red )& (int(element[1])>green) &( int(element[2])>blue)):
        result.append(element)
      else:
        result.append(tuple([255,255,255]))
  elif option == "OU": 
    for element in rgb:
      if((int(element[0])>red) | (int(element[1])>green) | (int(element[2])>blue)):
        result.append(element)
      else:
        result.append(tuple([255,255,255]))

  print(result)


In [48]:
thresholding(rgb,62,20,30,"ET")

[('62', '29', '64'), ('63', '31', '64'), ('63', '34', '64'), ('65', '30', '60'), ('66', '27', '59'), ('63', '31', '62'), ('61', '31', '62'), ('63', '30', '61'), ('63', '30', '59'), ('64', '31', '58'), ('64', '30', '61'), ('61', '30', '58'), ('65', '30', '56'), ('64', '32', '59'), ('61', '31', '59'), ('62', '30', '60'), ('63', '30', '62'), ('65', '32', '64'), ('65', '32', '61'), ('64', '29', '58'), ('61', '29', '57'), ('64', '30', '56'), ('65', '29', '58'), ('64', '30', '61'), ('66', '30', '61'), ('71', '31', '66'), ('67', '33', '67'), ('65', '33', '63'), ('64', '35', '62'), ('62', '32', '62'), ('68', '31', '64'), ('68', '35', '66'), ('70', '31', '65'), ('70', '31', '63'), ('71', '33', '65'), ('70', '34', '68'), ('68', '33', '67'), ('74', '37', '69'), ('74', '37', '69'), ('70', '36', '69'), ('70', '36', '70'), ('71', '37', '67'), ('71', '36', '67'), ('72', '34', '69'), ('74', '35', '70'), ('70', '34', '70'), ('69', '34', '67'), ('70', '35', '65'), ('67', '34', '63'), ('67', '32', '65'),

# Automatic Threasholding

In [62]:
def hitogram_ppm(im):
  hist_red=np.zeros((255))
  for element in im:
    hist_red[int(element[0])] += 1
  hist_green=np.zeros((255))
  for element in im:
    hist_green[int(element[0])] += 1
  hist_blue=np.zeros((255))
  for element in im:
    hist_blue[int(element[0])] += 1
  index = range(0,255)

  histcum_red=np.zeros((255))
  histcum_green=np.zeros((255))
  histcum_blue=np.zeros((255))
  for i in range(0,254):
    histcum_red[i]=hist_red[i]+hist_red[i-1]
    histcum_green[i]=hist_green[i]+hist_green[i-1]
    histcum_blue[i]=hist_blue[i]+hist_blue[i-1]
  #plt.bar(index, histcum_red) ; plt.show()
  print(histcum_red)

In [64]:
hitogram_ppm(rgb)
seuil_red=input('donner selon le seuil red: ')
seuil_green=input('donner selon le seuil green: ')
seuil_blue=input('donner selon le seuil blue: ')
thresholding(rgb,seuil_red,seuil_green,seuil_blue)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  2.  1.  0.  0.  0.  2.  2.  0.
  4.  5.  2.  6.  8.  6.  7. 10. 18. 24. 27. 32. 31. 33. 40. 40. 51. 52.
 30. 18. 18. 12.  8.  6.  0.  1.  2.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

TypeError: ignored